# Week 5 Homework

In this homework we'll put what we learned about Spark in practice.

For this homework we will be using the FHV 2019-10 data found here. [FHV Data](https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2019-10.parquet)

### Question 1:
Install Spark and PySpark
Install Spark
Run PySpark
Create a local spark session
Execute spark.version
What's the output?

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local[*]") \
        .appName("test") \
        .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/03/10 17:42:22 WARN Utils: Your hostname, Yishan resolves to a loopback address: 127.0.1.1; using 172.23.55.174 instead (on interface eth0)
24/03/10 17:42:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/10 17:42:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.version

'3.5.1'

### Question 2
FHV October 2019

Read the October 2019 FHV into a Spark Dataframe with a schema as we did in the lessons.

Repartition the Dataframe to 6 partitions and save it to parquet.

What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches.

- 1MB
- **6MB**
- 25MB
- 87MB

In [5]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2019-10.parquet

--2024-03-10 17:57:05--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2019-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.64.72.181, 18.64.72.109, 18.64.72.32, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.64.72.181|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18344035 (17M) [application/x-www-form-urlencoded]
Saving to: ‘fhv_tripdata_2019-10.parquet’

fhv_tripdata_2019-1 100%[===================>]  17.49M  43.9MB/s    in 0.4s    

2024-03-10 17:57:06 (43.9 MB/s) - ‘fhv_tripdata_2019-10.parquet’ saved [18344035/18344035]



In [6]:
!wc -l fhv_tripdata_2019-10.parquet

49967 fhv_tripdata_2019-10.parquet


In [7]:
df = spark.read \
    .option("header", "true") \
    .parquet("fhv_tripdata_2019-10.parquet")

In [8]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|       264.0|       264.0|   NULL|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|       264.0|       264.0|   NULL|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|       264.0|       264.0|   NULL|                B00014|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|       264.0|       264.0|   NULL|                B00014|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|       264.0|       264.0|   NULL|                B00014|
|     B00021         |2019-10-01 00:00:4

In [9]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampNTZType(), True), StructField('dropOff_datetime', TimestampNTZType(), True), StructField('PUlocationID', DoubleType(), True), StructField('DOlocationID', DoubleType(), True), StructField('SR_Flag', IntegerType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [10]:
# partition the file
df = df.repartition(6)

In [11]:
df.write.parquet("fhvhv/2019/10/", mode="overwrite")

In [12]:
!du -sh /home/labber/dtc-de/WEEK5-PYSPARK/fhvhv/2019/10/part-00000-61109c35-0b3b-401c-acf3-c060d533c1cd-c000.snappy.parquet

5.7M	/home/labber/dtc-de/WEEK5-PYSPARK/fhvhv/2019/10/part-00000-61109c35-0b3b-401c-acf3-c060d533c1cd-c000.snappy.parquet


### Question 3:
Count records

How many taxi trips were there on the 15th of October?

Consider only trips that started on the 15th of October.

- 108,164
- 12,856
- 452,470
- **62,610**

In [13]:
df = spark.read.parquet("fhvhv/2019/10/")

In [14]:
df.show(5)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02293|2019-10-21 14:00:37|2019-10-21 14:42:58|        33.0|        71.0|   NULL|                B02293|
|              B02677|2019-10-19 04:39:20|2019-10-19 04:41:26|       264.0|        76.0|   NULL|                B02677|
|              B02509|2019-10-18 11:22:44|2019-10-18 12:00:49|       264.0|       213.0|   NULL|                B02872|
|              B01145|2019-10-01 15:46:59|2019-10-01 15:59:10|       264.0|       159.0|   NULL|                B02887|
|              B01259|2019-10-03 16:01:56|2019-10-03 16:18:19|       264.0|       217.0|   NULL|                B01259|
+--------------------+------------------

In [15]:
from pyspark.sql import functions as F

df_filtered = df.filter(F.to_date(df.pickup_datetime) == "2019-10-15")
row_count = df_filtered.count()
row_count

62629

### Question 4:

Longest trip for each day

What is the length of the longest trip in the dataset in hours?

- **631,152.50 Hours**
- 243.44 Hours
- 7.68 Hours
- 3.32 Hours

In [21]:
time_diff = F.to_timestamp(df.dropOff_datetime) - F.to_timestamp(df.pickup_datetime)
df = df.withColumn('time_diff', time_diff)
max_diff = df.select(F.max('time_diff')).collect()[0][0]
max_diff

datetime.timedelta(days=26298, seconds=1800)

### Question 5:
User Interface

Spark’s User Interface which shows the application's dashboard runs on which local port?

- 80
- 443
- **4040**
- 8080

### Question 6:
Least frequent pickup location zone

Load the zone lookup data into a temp view in Spark
Zone Data

Using the zone lookup data and the FHV October 2019 data, what is the name of the LEAST frequent pickup location Zone?

- East Chelsea
- **Jamaica Bay**
- Union Sq
- Crown Heights North

In [27]:
df_zones = spark.read.parquet("zones/part-00000-307330ea-7116-4920-a709-6a257fc23500-c000.snappy.parquet")
df_zones.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [28]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropOff_datetime: timestamp_ntz (nullable = true)
 |-- PUlocationID: double (nullable = true)
 |-- DOlocationID: double (nullable = true)
 |-- SR_Flag: integer (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)
 |-- time_diff: interval day to second (nullable = true)



In [29]:
df_result = df.join(df_zones, df["PULocationID"] == df_zones["LocationID"], "left")
df_result.show(5)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+--------------------+----------+--------+----------------+------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|           time_diff|LocationID| Borough|            Zone|service_zone|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+--------------------+----------+--------+----------------+------------+
|              B02293|2019-10-21 14:00:37|2019-10-21 14:42:58|        33.0|        71.0|   NULL|                B02293|INTERVAL '0 00:42...|        33|Brooklyn|Brooklyn Heights|   Boro Zone|
|              B02677|2019-10-19 04:39:20|2019-10-19 04:41:26|       264.0|        76.0|   NULL|                B02677|INTERVAL '0 00:02...|       264| Unknown|              NV|         N/A|
|              B02509|2019-10-18 11:22:44|201

In [32]:
df_result.registerTempTable('result')

/home/labber/.local/lib/python3.10/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [34]:
least_pickup = spark.sql("SELECT Zone, count(*) as total FROM result GROUP BY Zone ORDER BY total ASC LIMIT 1").show()


+-----------+-----+
|       Zone|total|
+-----------+-----+
|Jamaica Bay|    1|
+-----------+-----+

